In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFECV
import warnings
warnings.filterwarnings('ignore')

# Set a consistent style for all plots
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Load the dataset
df_yield = pd.read_csv("/Users/ash/CropYield_estimation /DATASET/yield.csv")
df_temp = pd.read_csv("/Users/ash/CropYield_estimation /DATASET/temp.csv")
df_rainfall = pd.read_csv("/Users/ash/CropYield_estimation /DATASET/rainfall.csv")
df_pesticides = pd.read_csv("/Users/ash/CropYield_estimation /DATASET/pesticides.csv")
df_yield_df = pd.read_csv("/Users/ash/CropYield_estimation /DATASET/yield_df.csv")

# Data preprocessing
print("Data Preprocessing...")
if 'Unnamed: 0' in df_yield_df.columns:
    df_yield_df.drop(['Unnamed: 0'], axis=1, inplace=True)
df_rainfall.rename(columns={' Area':'Area'}, inplace=True)
df_temp.rename(columns={'year':'Year', 'country':'Area'}, inplace=True)

# Merge datasets
df_temprain = pd.merge(df_rainfall, df_temp, on=['Year','Area'])
df_trp = pd.merge(df_temprain, df_pesticides, on=['Year','Area'])
yield_df = pd.merge(df_yield_df, df_yield, on=['Year','Area','Item'])

# Clean and prepare the final dataset
Yield_final_data = yield_df.copy()
columns_to_drop = ['Area Code', 'Year Code', 'Domain', 'Domain Code', 'Item Code', 
                   'Element', 'Element Code', 'Unit', 'Value']
Yield_final_data.drop(columns_to_drop, axis=1, inplace=True)

# Split 'Item' column with multiple crops
Yield_final_data['Item'] = Yield_final_data['Item'].str.split(', ')
Yield_final_data = Yield_final_data.explode('Item').reset_index(drop=True)

print(f"Dataset shape after preprocessing: {Yield_final_data.shape}")

# Exploratory Data Analysis
print("\nExploratory Data Analysis...")

# Distribution of target variable
plt.figure(figsize=(10, 6))
sns.histplot(Yield_final_data['hg/ha_yield'], kde=True)
plt.title('Distribution of Crop Yield')
plt.xlabel('Yield (hg/ha)')
plt.ylabel('Frequency')
plt.savefig('yield_distribution.png')
plt.close()

# Log transform of target for better distribution
plt.figure(figsize=(10, 6))
sns.histplot(np.log1p(Yield_final_data['hg/ha_yield']), kde=True)
plt.title('Distribution of Log-transformed Crop Yield')
plt.xlabel('Log(Yield+1)')
plt.ylabel('Frequency')
plt.savefig('log_yield_distribution.png')
plt.close()

# Yield by crop type
plt.figure(figsize=(14, 8))
sns.boxplot(x='Item', y='hg/ha_yield', data=Yield_final_data)
plt.title('Yield Distribution by Crop Type')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('yield_by_crop.png')
plt.close()

# Correlation between numerical features
numerical_features = ['Year', 'average_rain_fall_mm_per_year', 'avg_temp', 'pesticides_tonnes', 'hg/ha_yield']
correlation_matrix = Yield_final_data[numerical_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.close()

# Feature Engineering
print("\nFeature Engineering...")

# 1. Create binary features for each crop type
crop_dummies = pd.get_dummies(Yield_final_data['Item'], prefix='crop')
Yield_final_data = pd.concat([Yield_final_data, crop_dummies], axis=1)

# 2. Create interaction features
Yield_final_data['temp_rain_interaction'] = Yield_final_data['avg_temp'] * Yield_final_data['average_rain_fall_mm_per_year']
Yield_final_data['pesticide_per_temp'] = Yield_final_data['pesticides_tonnes'] / (Yield_final_data['avg_temp'] + 1e-6)
Yield_final_data['rain_squared'] = Yield_final_data['average_rain_fall_mm_per_year'] ** 2
Yield_final_data['temp_squared'] = Yield_final_data['avg_temp'] ** 2

# 3. Log transform skewed features
Yield_final_data['log_pesticides'] = np.log1p(Yield_final_data['pesticides_tonnes'])
Yield_final_data['log_yield'] = np.log1p(Yield_final_data['hg/ha_yield'])

# 4. Create temporal features
Yield_final_data['year_squared'] = Yield_final_data['Year'] ** 2
Yield_final_data['year_normalized'] = (Yield_final_data['Year'] - Yield_final_data['Year'].min()) / (Yield_final_data['Year'].max() - Yield_final_data['Year'].min())

# Prepare data for modeling
y = Yield_final_data['log_yield']  # Using log-transformed yield
X = Yield_final_data.drop(['hg/ha_yield', 'Item', 'Area', 'log_yield'], axis=1)

# Identify categorical and numerical columns
categorical_cols = [c for c in X.columns if c.startswith('crop_')]
numerical_cols = [c for c in X.columns if c not in categorical_cols]

print(f"Number of features: {X.shape[1]} (Numerical: {len(numerical_cols)}, Categorical: {len(categorical_cols)})")

# Feature selection using a baseline model to identify important features
print("\nPerforming Feature Selection...")

# Create preprocessing pipeline for feature selection
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Split data for feature selection
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use Recursive Feature Elimination with Cross-Validation for feature selection
base_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit preprocessor
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Get feature names after preprocessing
numeric_features = numerical_cols
categorical_features = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
all_features = np.concatenate([numeric_features, categorical_features.tolist()])

# Train Random Forest to get feature importances
rf_selector = RandomForestRegressor(n_estimators=100, random_state=42)
rf_selector.fit(X_train_preprocessed, y_train)

# Get feature importances
feature_importances = pd.DataFrame({
    'Feature': all_features,
    'Importance': rf_selector.feature_importances_
}).sort_values('Importance', ascending=False)

# Visualize feature importances
plt.figure(figsize=(12, 10))
sns.barplot(x='Importance', y='Feature', data=feature_importances.head(15))
plt.title('Top 15 Feature Importances')
plt.tight_layout()
plt.savefig('feature_importances.png')
plt.close()

# Select top features
top_n_features = 15  # Adjust based on feature importance results
top_features = feature_importances.head(top_n_features)['Feature'].tolist()

print(f"Selected top {top_n_features} features:")
for i, feature in enumerate(top_features):
    print(f"{i+1}. {feature}")

# Extract indices of top features from all features
top_indices = [np.where(all_features == feature)[0][0] for feature in top_features]

# Create filtered datasets with only top features
X_train_selected = X_train_preprocessed[:, top_indices]
X_test_selected = X_test_preprocessed[:, top_indices]

# Define models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.001),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    'SVR': SVR(kernel='rbf', C=1.0, epsilon=0.1)
}

# Model evaluation function with cross-validation
def evaluate_model(model, X, y, cv=5):
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    r2_scores = cross_val_score(model, X, y, cv=cv, scoring='r2')
    
    return {
        'RMSE': rmse_scores.mean(),
        'RMSE_std': rmse_scores.std(),
        'R2': r2_scores.mean(),
        'R2_std': r2_scores.std()
    }

# Evaluate each model with cross-validation
print("\nModel Evaluation with Cross-Validation:")
cv_results = {}

for name, model in models.items():
    print(f"Evaluating {name}...")
    cv_result = evaluate_model(model, X_train_selected, y_train)
    cv_results[name] = cv_result
    print(f"  RMSE: {cv_result['RMSE']:.3f} ± {cv_result['RMSE_std']:.3f}")
    print(f"  R2: {cv_result['R2']:.3f} ± {cv_result['R2_std']:.3f}")

# Visualize cross-validation results
cv_df = pd.DataFrame.from_dict(cv_results, orient='index')
cv_df['Model'] = cv_df.index

# Plot RMSE scores
plt.figure(figsize=(12, 6))
sns.barplot(x='Model', y='RMSE', data=cv_df)
plt.errorbar(x=range(len(cv_df)), y=cv_df['RMSE'], yerr=cv_df['RMSE_std'], fmt='none', color='black')
plt.title('Cross-Validation RMSE by Model')
plt.xlabel('Model')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('cv_rmse_comparison.png')
plt.close()

# Plot R2 scores
plt.figure(figsize=(12, 6))
sns.barplot(x='Model', y='R2', data=cv_df)
plt.errorbar(x=range(len(cv_df)), y=cv_df['R2'], yerr=cv_df['R2_std'], fmt='none', color='black')
plt.title('Cross-Validation R² Score by Model')
plt.xlabel('Model')
plt.ylabel('R² Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('cv_r2_comparison.png')
plt.close()

# Find the best model from cross-validation
best_model_name = cv_df.sort_values('RMSE')['Model'].iloc[0]
print(f"\nBest model based on cross-validation: {best_model_name}")

# Train all models on the complete training set for final evaluation
results = {}
for name, model in models.items():
    print(f"Training {name} on full training set...")
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    
    # Convert back from log scale
    y_test_exp = np.expm1(y_test)
    y_pred_exp = np.expm1(y_pred)
    
    mse = mean_squared_error(y_test_exp, y_pred_exp)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_exp, y_pred_exp)
    mae = mean_absolute_error(y_test_exp, y_pred_exp)
    
    results[name] = {
        'RMSE': rmse,
        'R2': r2,
        'MAE': mae,
        'Model': model,
        'Predictions': y_pred_exp
    }
    
    print(f"  RMSE: {rmse:.3f}")
    print(f"  R2: {r2:.3f}")
    print(f"  MAE: {mae:.3f}")

# Visualize model performance comparison
metrics_df = pd.DataFrame({
    'Model': list(results.keys()),
    'RMSE': [results[model]['RMSE'] for model in results],
    'MAE': [results[model]['MAE'] for model in results],
    'R2': [results[model]['R2'] for model in results]
})

# Plot RMSE comparison
plt.figure(figsize=(12, 6))
sns.barplot(x='Model', y='RMSE', data=metrics_df.sort_values('RMSE'))
plt.title('Test Set RMSE by Model')
plt.xlabel('Model')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('test_rmse_comparison.png')
plt.close()

# Plot R2 comparison
plt.figure(figsize=(12, 6))
sns.barplot(x='Model', y='R2', data=metrics_df.sort_values('R2', ascending=False))
plt.title('Test Set R² Score by Model')
plt.xlabel('Model')
plt.ylabel('R² Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('test_r2_comparison.png')
plt.close()

# Find the best model from test set
best_model_name = min(results, key=lambda x: results[x]['RMSE'])
best_model = results[best_model_name]['Model']
print(f"\nBest model based on test set: {best_model_name}")

# Visualize actual vs predicted values for the best model
y_pred_best = results[best_model_name]['Predictions']
y_test_exp = np.expm1(y_test)

plt.figure(figsize=(10, 8))
plt.scatter(y_test_exp, y_pred_best, alpha=0.5)
plt.plot([y_test_exp.min(), y_test_exp.max()], [y_test_exp.min(), y_test_exp.max()], 'k--', lw=2)
plt.xlabel('Actual Yield')
plt.ylabel('Predicted Yield')
plt.title(f'Actual vs Predicted Crop Yield ({best_model_name})')
plt.tight_layout()
plt.savefig('actual_vs_predicted.png')
plt.close()

# Residual analysis
residuals = y_test_exp - y_pred_best
plt.figure(figsize=(10, 8))
plt.scatter(y_pred_best, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='-')
plt.xlabel('Predicted Yield')
plt.ylabel('Residuals')
plt.title(f'Residual Plot ({best_model_name})')
plt.tight_layout()
plt.savefig('residual_plot.png')
plt.close()

# Error distribution
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.xlabel('Prediction Error')
plt.title(f'Error Distribution ({best_model_name})')
plt.tight_layout()
plt.savefig('error_distribution.png')
plt.close()

# Prediction error by crop type
# Get original indices from test set
test_indices = y_test.index

# Create a dataframe for error analysis by category
error_df = pd.DataFrame({
    'Actual': y_test_exp,
    'Predicted': y_pred_best,
    'Error': residuals,
    'Item': Yield_final_data.loc[test_indices, 'Item'],
    'Year': Yield_final_data.loc[test_indices, 'Year']
})

# Error by crop type
plt.figure(figsize=(14, 8))
sns.boxplot(x='Item', y='Error', data=error_df)
plt.title('Prediction Error by Crop Type')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('error_by_crop.png')
plt.close()

# Model comparison - actual vs predicted scatter plots in one figure
plt.figure(figsize=(20, 15))
models_to_plot = ['Linear Regression', 'Random Forest', 'Gradient Boosting', 'XGBoost']


for i, model_name in enumerate(models_to_plot, 1):
    plt.subplot(2, 2, i)
    y_pred = results[model_name]['Predictions']
    plt.scatter(y_test_exp, y_pred, alpha=0.5)
    plt.plot([y_test_exp.min(), y_test_exp.max()], [y_test_exp.min(), y_test_exp.max()], 'k--', lw=2)
    plt.xlabel('Actual Yield')
    plt.ylabel('Predicted Yield')
    plt.title(f'{model_name} (RMSE: {results[model_name]["RMSE"]:.2f}, R²: {results[model_name]["R2"]:.3f})')

plt.tight_layout()
plt.savefig('model_comparison_plots.png')
plt.close()

print("\nAnalysis completed. All visualizations have been saved.")

Data Preprocessing...
Dataset shape after preprocessing: (31630, 7)

Exploratory Data Analysis...

Feature Engineering...
Number of features: 22 (Numerical: 11, Categorical: 11)

Performing Feature Selection...
Selected top 15 features:
1. crop_Potatoes_True
2. crop_Cassava_True
3. crop_Sweet potatoes_True
4. crop_Potatoes_False
5. temp_rain_interaction
6. pesticide_per_temp
7. average_rain_fall_mm_per_year
8. rain_squared
9. crop_Sorghum_False
10. temp_squared
11. log_pesticides
12. avg_temp
13. crop_Yams_True
14. pesticides_tonnes
15. crop_Soybeans_False

Model Evaluation with Cross-Validation:
Evaluating Linear Regression...
  RMSE: 0.585 ± 0.007
  R2: 0.697 ± 0.007
Evaluating Ridge Regression...
  RMSE: 0.585 ± 0.007
  R2: 0.697 ± 0.007
Evaluating Lasso Regression...
  RMSE: 0.585 ± 0.007
  R2: 0.697 ± 0.006
Evaluating Random Forest...
  RMSE: 0.453 ± 0.005
  R2: 0.819 ± 0.005
Evaluating Gradient Boosting...
  RMSE: 0.479 ± 0.007
  R2: 0.797 ± 0.005
Evaluating XGBoost...
  RMSE: 0.